In [19]:
import sys
import os
# sys.path.append(os.path.abspath('../models'))
dir_path = os.path.dirname("/Users/victorsu-ortiz/Desktop/Fe-Boryl_complexes/models/")
sys.path.append(dir_path)
from gcn_mse import run_gcn_mse 
from deephyper.problem import HpProblem
from deephyper.search.hps import CBO
from deephyper.evaluator import Evaluator
import ray

In [20]:
import torch

if torch.backends.mps.is_available():
    print("MPS is available!")
else:
    print("MPS is not available.")

MPS is available!


In [42]:
# Problem definition
hp = HpProblem()
hp.add_hyperparameter([True, False], "use_batch_norm")
hp.add_hyperparameter((16, 128), "batch_size")
hp.add_hyperparameter((3, 10), "patience")
hp.add_hyperparameter((0.1, 0.9), "scheduler_factor")
hp.add_hyperparameter((16, 256), "hidden_dim1")
hp.add_hyperparameter((16, 256), "hidden_dim2")
hp.add_hyperparameter((16, 256), "hidden_dim3")
hp.add_hyperparameter((16, 256), "hidden_dim4")
hp.add_hyperparameter((16, 256), "hidden_dim5")
hp.add_hyperparameter((16, 256), "hidden_dim6")
hp.add_hyperparameter((1, 2), "num_layers1")
hp.add_hyperparameter((1, 2), "num_layers2")
hp.add_hyperparameter((1, 2), "num_layers3")
hp.add_hyperparameter((1, 2), "num_layers4")
hp.add_hyperparameter((1, 2), "num_layers5")
hp.add_hyperparameter((1, 2), "num_layers6")
# if run_function == run_gat_mse:
#     hp.add_hyperparameter((1, 8), "num_heads1")  # Number of attention heads for layer 1
#     hp.add_hyperparameter((1, 8), "num_heads2")  # Number of attention heads for layer 2
#     hp.add_hyperparameter((1, 8), "num_heads3")  # Number of attention heads for layer 3
#     hp.add_hyperparameter((1, 8), "num_heads4")  # Number of attention heads for layer 4
#     hp.add_hyperparameter((1, 8), "num_heads5")  # Number of attention heads for layer 5
#     hp.add_hyperparameter((1, 8), "num_heads6")  # Number of attention heads for layer 6
hp.add_hyperparameter((1e-5, 1e-3, "log-uniform"), "lr")
hp.add_hyperparameter([200], "epochs")

OrdinalHyperparameter(name='epochs', default_value=200, meta=None, size=1, sequence=(200,), _contains_sequence_as_value=False)

In [45]:
# Ray initialization
ray.init(runtime_env={"working_dir": ".","py_modules": ["/Users/victorsu-ortiz/Desktop/Fe-Boryl_complexes/models/"]})

2024-07-29 13:36:17,377	INFO worker.py:1781 -- Started a local Ray instance.
2024-07-29 13:36:17,382	INFO packaging.py:530 -- Creating a file package for local directory '/Users/victorsu-ortiz/Desktop/Fe-Boryl_complexes/models/'.
2024-07-29 13:36:17,383	INFO packaging.py:358 -- Pushing file package 'gcs://_ray_pkg_8ac688f082205fb7.zip' (0.02MiB) to Ray cluster...
2024-07-29 13:36:17,384	INFO packaging.py:371 -- Successfully pushed file package 'gcs://_ray_pkg_8ac688f082205fb7.zip'.
2024-07-29 13:36:18,234	INFO packaging.py:530 -- Creating a file package for local directory '.'.
2024-07-29 13:36:18,710	INFO packaging.py:358 -- Pushing file package 'gcs://_ray_pkg_9bc771c0e65ddfe6.zip' (7.69MiB) to Ray cluster...
2024-07-29 13:36:18,722	INFO packaging.py:371 -- Successfully pushed file package 'gcs://_ray_pkg_9bc771c0e65ddfe6.zip'.


Python version:,3.12.4
Ray version:,2.33.0


In [46]:
run_default = ray.remote(num_cpus=1)(run_gcn_mse)
out = ray.get(run_default.remote())

TypeError: missing a required argument: 'hp'

In [44]:
ray.shutdown()

In [24]:


# Evaluator creation
print("Creation of the Evaluator...")
num_total_gpus_on_node = 3
evaluator = Evaluator.create(
    run_gcn_mse,  # Pass the function itself, not run_fun.remote
    method="ray",
    method_kwargs={
        "num_cpus": num_total_gpus_on_node,
        # "num_gpus": num_total_gpus_on_node,
        "num_cpus_per_task": 1,
        # "num_gpus_per_task": 1,
        "num_workers": 3,
    }
)

Creation of the Evaluator...


In [25]:
h = {
        'batch_size': 16,
        'patience': 10,
        'scheduler_factor': 0.8,
        'hidden_dim1': 32,
        'hidden_dim2': 64,
        'hidden_dim3': 128,
        'hidden_dim4': 64,
        'hidden_dim5': 32,
        'hidden_dim6': 16,
        'num_layers1': 2,
        'num_layers2': 2,
        'num_layers3': 2,
        'num_layers4': 2,
        'num_layers5': 2,
        'num_layers6': 2,
        'lr': 0.00005,
        'epochs': 200,
        'use_batch_norm': True
    }

run_gcn_mse(h)

Epoch 1 of job unknown, Validation Loss: 1.2802526
Epoch 2 of job unknown, Validation Loss: 1.2123943
Epoch 3 of job unknown, Validation Loss: 1.2540897
Epoch 4 of job unknown, Validation Loss: 1.1794041
Epoch 5 of job unknown, Validation Loss: 1.2231928
Epoch 6 of job unknown, Validation Loss: 1.0725513
Epoch 7 of job unknown, Validation Loss: 1.1790759
Epoch 8 of job unknown, Validation Loss: 1.1091026
Epoch 9 of job unknown, Validation Loss: 1.1160442
Epoch 10 of job unknown, Validation Loss: 0.9979223
Epoch 11 of job unknown, Validation Loss: 0.9870191
Epoch 12 of job unknown, Validation Loss: 0.9748496
Epoch 13 of job unknown, Validation Loss: 0.9726558
Epoch 14 of job unknown, Validation Loss: 0.8531585


KeyboardInterrupt: 

In [26]:
# Search creation
print("Creation of the search instance...")
search = CBO(
    hp,
    evaluator,
)

results = search.search(max_evals=25)
print(results)

Creation of the search instance...


RayTaskError(RuntimeError): [36mray::run_gcn_mse()[39m (pid=96559, ip=127.0.0.1)
RuntimeError: The remote function failed to import on the worker. This may be because needed library dependencies are not installed in the worker environment or cannot be found from sys.path ['/Users/victorsu-ortiz/Desktop/Fe-Boryl_complexes/.venv/lib/python3.12/site-packages/ray/thirdparty_files', '/Users/victorsu-ortiz/Desktop/Fe-Boryl_complexes/.venv/lib/python3.12/site-packages/ray/_private/workers', '/tmp/ray/session_2024-07-29_13-24-57_107739_95742/runtime_resources/py_modules_files/_ray_pkg_8ac688f082205fb7', '/tmp/ray/session_2024-07-29_13-24-57_107739_95742/runtime_resources/working_dir_files/_ray_pkg_1cb5967f8c5956d3', '/Users/victorsu-ortiz/miniconda3/lib/python312.zip', '/Users/victorsu-ortiz/miniconda3/lib/python3.12', '/Users/victorsu-ortiz/miniconda3/lib/python3.12/lib-dynload', '/Users/victorsu-ortiz/Desktop/Fe-Boryl_complexes/.venv/lib/python3.12/site-packages']:

[36mray::run_gcn_mse()